In [1]:
import sys
import os
import torch
sys.path.append(os.path.abspath("../"))
from base_experiment import ExperimentRunner

In [2]:
experiment = ExperimentRunner(
    experiment_name = "nba_fair_ac_main",
    seeds = [41],
    # seeds = [40, 41, 42],
    data_path = "dataset/NBA",
    log_dir="experiments/fair_ac/logs/nba_testing", 
    device=2,
    params=[{"lambda1": 1.0, "lambda2": 1.0}]
)

# after we set up the experiment, we can import the rest
from dataset import NBA
from models.gnn import WrappedGNNConfig
from models.fair.ac import FairAC, Trainer

In [3]:
# Load in the dataset
dataset = NBA(
    nodes_path=experiment.data_path / "nba.csv",
    edges_path=experiment.data_path / "nba_relationship.txt",
    embedding_path=experiment.data_path / "nba_embedding10.npy",
    feat_drop_rate=0.3,
    device=experiment.device
)

print(f"Loaded dataset with {dataset.graph.num_nodes()} nodes and {dataset.graph.num_edges()} edges")
print(f"Using feat_drop_rate: {dataset.feat_drop_rate}")

Loaded dataset with 403 nodes and 21645 edges
Using feat_drop_rate: 0.3


In [4]:
# Create downstream GNN config
gnn_config = WrappedGNNConfig(
    hidden_dim=128,
    kind="GCN",
    lr=1e-3,
    weight_decay=1e-5,
    kwargs={"dropout": 0.5},
)

In [5]:
for (seed, log_dir, device, params) in experiment.runs():
    print("===========================")
    print(f"Running {experiment.experiment_name} using seed {seed}")
    print(f"Log directory: {log_dir}")
    print(f"Params: {params}")
    print("===========================")
    
    # Create FairAC model
    fair_ac = FairAC(
        feature_dim=dataset.features.shape[1],
        transformed_feature_dim=128,
        emb_dim=dataset.embeddings.shape[1],
        attn_vec_dim=128,
        attn_num_heads=1,
        dropout=0.5,
        num_sensitive_classes=1,
    ).to(experiment.device)
    print(f"Created FairAC model with {1} sensitive class")
        
    # Create FairAC trainer
    trainer = Trainer(
        ac_model=fair_ac,
        lambda1=params["lambda1"],
        lambda2=params["lambda2"],
        dataset=dataset,
        device=experiment.device,
        gnn_config=gnn_config,
        log_dir=log_dir,
        min_acc=0.65,
        min_roc=0.69,
    )
    print(f"Created trainer with {'GCN'} model, using log_dir: {log_dir}")

    print("Starting pre-training phase")
    # Run pre-training
    trainer.pretrain(epochs=200)
    print("Finished pretraining")
    
    # Main training loop, with GNN validation
    print("Starting main training...")
    trainer.train(val_start_epoch=800, val_epoch_interval=200, epochs=2800)


SETTING SEED TO: 41
Running nba_fair_ac_main using seed 41
Log directory: /home/fact21/fact_refactor/experiments/fair_ac/logs/nba_testing/nba_fair_ac_main_41_lambda1_1.0_lambda2_1.0
Params: {'lambda1': 1.0, 'lambda2': 1.0}
Created FairAC model with 1 sensitive class
Created trainer with GCN model, using log_dir: /home/fact21/fact_refactor/experiments/fair_ac/logs/nba_testing/nba_fair_ac_main_41_lambda1_1.0_lambda2_1.0
Starting pre-training phase


  0%|          | 0/200 [00:00<?, ?it/s]

Finished pretraining
Starting main training...


  0%|          | 0/2800 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

[2600:731] Found new best fairness of 0.002036


  0%|          | 0/1000 [00:00<?, ?it/s]

[2799:906] Found new best fairness of 0.001691
Finished training!
Best epoch: 2799

Best fair model:
	acc: 0.6714
	roc: 0.7441
	parity: 0.0002
	equality: 0.0015
	consistency: 0.0264

Best acc model:
	acc: 0.7465
	roc: 0.7626
	parity: 0.0399
	equality: 0.0798

Best auc model:
	acc: 0.5587
	roc: 0.7811
	parity: 0.0023
	equality: 0.0365
